In [1]:
import mysql.connector
import psycopg2

In [2]:
## Connecting MYSQL

connection = mysql.connector.connect(
    user = 'root',
    password = 'aW@nN2001',
    host = '127.0.0.1',
    database = 'salesp'
)

In [5]:
## Connecting Postgresql

dsn_hostname = '127.0.0.1'
dsn_user='postgres' 
dsn_pwd ='awana'  
dsn_port ="5432"
dsn_database ="salespb" 

# create connection

conn = psycopg2.connect(
   database=dsn_database, 
   user=dsn_user,
   password=dsn_pwd,
   host=dsn_hostname, 
   port= dsn_port
)

In [6]:
## MySQL cursor

cursorm = connection.cursor()

## Postgresql cursor
cursorp = conn.cursor()

In [16]:
def products():
    SQL = """
        SELECT * FROM products;
    """

    cursorp.execute(SQL)

    rows = cursorp.fetchall()

    for r in rows:
        print(r)



def sales_data():
    SQL = """
        SELECT * FROM sales_data limit 5;
    """

    cursorp.execute(SQL)

    rows = cursorp.fetchall()

    for r in rows:
        print(r)

In [17]:
products()
print()
sales_data()

(1, 'Television', 'Electronics')
(2, 'Laptop', 'Electronics')
(3, 'Mobile', 'Electronics')
(5, 'Mobile', 'Electronics')
(6, 'Mobile', 'Electronics')

(104, 8972, 58516, Decimal('4277'), 5, datetime.datetime(2017, 11, 22, 3, 53, 23))
(105, 7905, 59794, Decimal('542'), 2, datetime.datetime(2017, 11, 22, 3, 53, 24))
(106, 6295, 86874, Decimal('2706'), 2, datetime.datetime(2017, 11, 22, 3, 53, 25))
(107, 6665, 56517, Decimal('4821'), 1, datetime.datetime(2017, 11, 22, 3, 53, 26))
(108, 8851, 23457, Decimal('4612'), 3, datetime.datetime(2017, 11, 22, 3, 53, 27))


In [29]:
## Getting the last row id from postgresql database

def get_last_rowid():
    SQL = """
        SELECT rowid 
        FROM sales_data 
        ORDER BY rowid DESC
        LIMIT 1;
    """

    cursorp.execute(SQL)
    
    rows = cursorp.fetchone()

    return rows[0]

In [30]:
last_row_id = get_last_rowid()
print("Last row id on production datawarehouse = ", last_row_id)

Last row id on production datawarehouse =  12289


In [31]:
# List out all records in MySQL database with rowid greater than the one on the Data warehouse
# The function get_latest_records must return a list of all records that have a rowid greater
# than the last_row_id in the sales_data table in the sales database on the MySQL staging data 
# warehouse.

def get_latest_records(rd):
    SQL = f"""
        SELECT * FROM sales_data
        WHERE rowid > {rd};
    """

    cursorm.execute(SQL)

    rows = cursorm.fetchall()

    return rows


In [35]:
new_records = get_latest_records(12289)

In [36]:
print("New rows on staging datawarehouse = ", len(new_records))

New rows on staging datawarehouse =  1650


In [44]:
def rollback():
    cursorp.execute('rollback;')

In [47]:
rollback()

In [50]:

def insert_records(records):
    try:
        for r in records:
            SQL = """
                INSERT INTO sales_data (rowid, product_id, customer_id, price, quantity, timestamp)
                VALUES (%s, %s, %s, %s, %s, NOW())
            """
            cursorp.execute(SQL, (r[0], r[1], r[2], 0, r[3]))
    
    except psycopg2.Error as e:
        print("Error inserting records:", e)
        cursorp.connection.rollback()

insert_records(new_records)


In [54]:
def sales_data():
    SQL = """
        SELECT * FROM sales_data;
    """

    cursorp.execute(SQL)

    rows = cursorp.fetchall()

    return rows

In [55]:
new_data = sales_data()
print("New rows inserted into production datawarehouse = ", len(new_data))

New rows inserted into production datawarehouse =  13836


In [57]:
conn.close()    ## Disconnect from Postgresql

In [59]:
def close_cursor(cursor):
    try:
        cursorm.close()
    except mysql.connector.Error as e:
        print("Error closing cursor:", e)

# Example usage
close_cursor(cursorm)


Error closing cursor: Unread result found


In [60]:
def close_cursor(cursor):
    try:
        # Fetch any remaining rows to ensure all results are consumed
        cursor.fetchall()
    except mysql.connector.Error as e:
        # Handle any errors while fetching remaining rows
        print("Error fetching remaining rows:", e)

    try:
        # Close the cursor
        cursor.close()
    except mysql.connector.Error as e:
        # Handle any errors while closing the cursor
        print("Error closing cursor:", e)

# Example usage
close_cursor(cursorm)
